In [153]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')

import torch
import plotly.express as px
import src.nanda_plot
from src.nanda_plot import imshow_reversed, imshow
import pandas as pd
import numpy as np

In [319]:
data = torch.load("../results/locate_mechanism/gpt2_full_result.pt", map_location=torch.device('cpu'))

## Plot target win (200 examples)

In [321]:

imshow_reversed(
    data["mean"]["pos"]["per_block"].mean(0),
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    # x=[f"{tok}_{i}" for i, tok in enumerate(data["mean"]["pos"]["example_str_token"])],
)
imshow_reversed(
    data["var"]["pos"]["per_block"].mean(0),
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block (MAD)",
    xaxis="Position",
    yaxis="Layer",
    # x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)

### Subsample

In [316]:
imshow_reversed(
    data["pos"]["patch_per_block"]["mean"][0,:,:,:],
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)

KeyError: 'pos'

In [292]:
imshow_reversed(
    data["pos"]["patch_per_block"]["mean"][1,:,:,:],
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)

## Plot copy win (200 examples)

In [299]:
imshow(
    data["neg"]["patch_attn_head_out"]["mean"].mean(0),
    yaxis="Layer", 
    xaxis="Head", 
    title = "Attention heads"
)
imshow(
    data_var["neg"]["patch_attn_head_out"]["mean"].mean(0),
    yaxis="Layer", 
    xaxis="Head", 
    title = "Attention heads (MAD)",
    # zmax=1,
    # zmin=-1
)
imshow(
    data["neg"]["patch_per_block"]["mean"].mean(0),
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)
imshow(
    data_var["neg"]["patch_per_block"]["mean"].mean(0),
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block (MAD)",
    xaxis="Position",
    yaxis="Layer",
    # zmax= 6,
    # zmin=-6,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["neg"]["example_str_token"])],
)

### Subsample

In [294]:
imshow(
    data["neg"]["patch_per_block"]["mean"][0,:,:,:],
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)

In [295]:
imshow(
    data["neg"]["patch_per_block"]["mean"][1,:,:,:],
    facet_col=0,
    facet_labels=["Residual Stream", "Attn Output", "MLP Output"],
    title="Activation Patching Per Block",
    xaxis="Position",
    yaxis="Layer",
    # zmax=1,
    # zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(data["pos"]["example_str_token"])],
)